# Weight & Volume Estimation Experiment

This notebook experiments with AI-based weight/volume estimation using:
- **BigQuery** for data
- **OpenAI GPT-4o-mini** for estimation
- **Visualization** for error analysis

## Experiment Approaches
1. **Big Error Items** - Focus on items with high estimation error
2. **Uniformity Check** - Same products, multiple orders

## 1. Setup & Authentication

In [ ]:
# Install dependencies
!pip install -q google-cloud-bigquery openai pandas matplotlib seaborn tqdm

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

print("✅ Google Cloud authenticated")

In [ ]:
# Setup BigQuery client
from google.cloud import bigquery
import pandas as pd

PROJECT_ID = "sazoshop"  # Your GCP project ID
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ BigQuery client ready (project: {PROJECT_ID})")

In [ ]:
# Setup OpenAI client
from openai import OpenAI
import os
from google.colab import userdata

# Option 1: Use Colab Secrets (recommended)
# Go to: Runtime > Secrets > Add OPENAI_API_KEY
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except:
    # Option 2: Manual input (less secure)
    OPENAI_API_KEY = input("Enter your OpenAI API key: ")

openai_client = OpenAI(api_key=OPENAI_API_KEY)
print("✅ OpenAI client ready")

## 2. Load Data from BigQuery

In [ ]:
# Query 1: High Error Items (Weight)
# Items with highest weight estimation error

QUERY_HIGH_ERROR_WEIGHT = """
SELECT
  oid.order_item_order_id,
  oid.order_item_title_origin AS product_title,
  oid.order_item_product_version_info_category AS category,
  ARRAY_TO_STRING(oid.order_item_product_version_thumbnail_urls, '|') AS thumbnail_urls,
  
  -- Actual vs Estimated
  kse.actual_weight AS actual_weight_kg,
  SAFE_CAST(JSON_EXTRACT_SCALAR(oid.order_item_product_version_extra, '$.weight') AS FLOAT64) AS ai_weight_kg,
  
  -- Error rate
  ABS(SAFE_CAST(JSON_EXTRACT_SCALAR(oid.order_item_product_version_extra, '$.weight') AS FLOAT64) - kse.actual_weight) / kse.actual_weight AS weight_error_rate,
  
  kse.dimensions AS actual_dimensions

FROM `sazoshop.firestore_snapshot.v2_order_items` oid
INNER JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
  ON oid.order_item_order_id = kse.order_id

WHERE
  kse.actual_weight > 0 AND kse.actual_weight < 50
  AND kse.dimensions IS NOT NULL
  AND REGEXP_CONTAINS(kse.dimensions, r'^\\d+\\.?\\d*x\\d+\\.?\\d*x\\d+\\.?\\d*$')
  AND JSON_EXTRACT_SCALAR(oid.order_item_product_version_extra, '$.weight') IS NOT NULL
  AND oid.order_item_title_origin IS NOT NULL

ORDER BY weight_error_rate DESC
LIMIT 100
"""

df = client.query(QUERY_HIGH_ERROR_WEIGHT).to_dataframe()
print(f"✅ Loaded {len(df)} high-error items")
df.head()

In [ ]:
# Quick data exploration
print("=== Data Summary ===")
print(f"Total items: {len(df)}")
print(f"\nWeight Error Rate:")
print(df['weight_error_rate'].describe())
print(f"\nActual Weight (kg):")
print(df['actual_weight_kg'].describe())

In [ ]:
# Display sample images
from IPython.display import display, HTML, Image
import requests
from io import BytesIO

def show_product_sample(row):
    """Display product info with image"""
    urls = row['thumbnail_urls'].split('|') if row['thumbnail_urls'] else []
    img_html = ""
    if urls:
        img_html = f'<img src="{urls[0]}" style="max-width:150px; max-height:150px;">'
    
    html = f"""
    <div style="border:1px solid #ccc; padding:10px; margin:5px; display:inline-block; width:300px;">
        {img_html}
        <p><b>{row['product_title'][:50]}...</b></p>
        <p>Category: {row['category']}</p>
        <p>Actual: {row['actual_weight_kg']:.2f} kg | AI: {row['ai_weight_kg']:.2f} kg</p>
        <p>Error: {row['weight_error_rate']*100:.1f}%</p>
    </div>
    """
    return html

# Show top 6 high-error items
html_output = "<h3>Top High-Error Items</h3><div>"
for _, row in df.head(6).iterrows():
    html_output += show_product_sample(row)
html_output += "</div>"
display(HTML(html_output))

## 3. Define Estimation Prompt

In [ ]:
# System prompt (from prompts/weight-volume.system.txt)
SYSTEM_PROMPT = """You are a bot responsible for estimating the volume and weight of a product based on its data, focusing on packed volume estimation for items that can be folded, stacked, or compressed.

# Instructions
You will receive the following product data from the user:
- Title
- Category
- Image (if provided)

Use the text data and the image (if provided) to generate the most appropriate volume and weight for the product entered by the user.

# Estimation Process
Estimate the volume and weight based on the product's category, description, and image analysis. For items that can be folded, stacked, or compressed (e.g., clothing, towels, bedding, bags), estimate a reduced packed volume based on typical packing practices.

- **Clothing Items:** Use a standard packed volume of 3x15x15 cm and weight of 0.5 kg.
- **Foldable Items (e.g., towels, bags):** Estimate packed volume based on 50% reduction from their actual size if details indicate foldability or compressibility.
- **Stackable Items (e.g., boxes, storage containers):** Estimate packed volume considering stacking with a 30% volume reduction.
- **Non-Foldable/Non-Compressible Items:** Use actual dimensions without reduction.
- **Papers:** If the product can be reduced in volume by folding or rolling, take that into account and estimate the volume.

# Output Format
Provide the estimated weight and volume (including packed volume for foldable/compressible items) in JSON format. The output should include the volume in cm (width, length, depth), the weight in kg, and a brief reason for the estimate.

# Cautions
!Do not describe the reasoning process.
!Keep in mind that foldable or stackable sizes should be standardized for packaging.
!Answers should be in JSON format only.

# Example Output
{
  "volume": "20x40x2",
  "packed_volume": "10x20x2",
  "weight": 0.6,
  "reason": "Based on foldability and image analysis."
}
"""

print("✅ System prompt loaded")

## 4. Estimation Function

In [ ]:
import json
import time

def estimate_weight_volume(row, use_image=True):
    """
    Call OpenAI API to estimate weight and volume.
    Supports multimodal (text + image).
    """
    # Build user message content
    user_content = []
    
    # Text part
    text_prompt = f"""Title: {row['product_title']}
Category: {row['category']}"""
    user_content.append({"type": "text", "text": text_prompt})
    
    # Image part (if available and enabled)
    if use_image and row['thumbnail_urls']:
        image_url = row['thumbnail_urls'].split('|')[0]  # First image
        user_content.append({
            "type": "image_url",
            "image_url": {"url": image_url}
        })
    
    try:
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_content}
            ],
            temperature=0.01,
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        return {
            "success": True,
            "weight": result.get("weight"),
            "volume": result.get("volume"),
            "packed_volume": result.get("packed_volume"),
            "reason": result.get("reason")
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Test with first row
test_result = estimate_weight_volume(df.iloc[0])
print("Test result:")
print(json.dumps(test_result, indent=2))

## 5. Run Batch Estimation

In [ ]:
from tqdm import tqdm
import time

# Limit for testing (set to len(df) for full run)
LIMIT = 20

results = []
for idx, row in tqdm(df.head(LIMIT).iterrows(), total=LIMIT, desc="Estimating"):
    result = estimate_weight_volume(row, use_image=True)
    result['order_id'] = row['order_item_order_id']
    result['actual_weight_kg'] = row['actual_weight_kg']
    result['old_ai_weight_kg'] = row['ai_weight_kg']
    result['actual_dimensions'] = row['actual_dimensions']
    results.append(result)
    time.sleep(0.5)  # Rate limiting

results_df = pd.DataFrame(results)
print(f"\n✅ Completed {len(results_df)} estimations")
results_df.head()

## 6. Calculate Error Metrics

In [ ]:
# Filter successful results
success_df = results_df[results_df['success'] == True].copy()
print(f"Successful: {len(success_df)} / {len(results_df)}")

# Calculate new weight (handle None)
success_df['new_weight_kg'] = pd.to_numeric(success_df['weight'], errors='coerce')

# Calculate error rates
success_df['old_error_rate'] = abs(success_df['old_ai_weight_kg'] - success_df['actual_weight_kg']) / success_df['actual_weight_kg']
success_df['new_error_rate'] = abs(success_df['new_weight_kg'] - success_df['actual_weight_kg']) / success_df['actual_weight_kg']

# Summary
print("\n=== Error Rate Comparison ===")
print(f"Old AI Mean Error: {success_df['old_error_rate'].mean()*100:.1f}%")
print(f"New AI Mean Error: {success_df['new_error_rate'].mean()*100:.1f}%")
print(f"\nImprovement: {(success_df['old_error_rate'].mean() - success_df['new_error_rate'].mean())*100:.1f}%")

## 7. Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Error Rate Distribution
ax1 = axes[0]
sns.kdeplot(success_df['old_error_rate'], label='Old AI', color='steelblue', fill=True, ax=ax1)
sns.kdeplot(success_df['new_error_rate'], label='New AI', color='tomato', fill=True, ax=ax1)
ax1.set_title('Error Rate Distribution')
ax1.set_xlabel('Error Rate')
ax1.legend()

# 2. Predicted vs Actual (New)
ax2 = axes[1]
ax2.scatter(success_df['actual_weight_kg'], success_df['new_weight_kg'], alpha=0.6, label='New AI')
ax2.scatter(success_df['actual_weight_kg'], success_df['old_ai_weight_kg'], alpha=0.4, label='Old AI', marker='x')
max_val = max(success_df['actual_weight_kg'].max(), success_df['new_weight_kg'].max())
ax2.plot([0, max_val], [0, max_val], '--', color='gray', label='Perfect')
ax2.set_title('Predicted vs Actual Weight')
ax2.set_xlabel('Actual Weight (kg)')
ax2.set_ylabel('Predicted Weight (kg)')
ax2.legend()

# 3. Improvement per item
ax3 = axes[2]
improvement = success_df['old_error_rate'] - success_df['new_error_rate']
colors = ['green' if x > 0 else 'red' for x in improvement]
ax3.bar(range(len(improvement)), improvement, color=colors)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_title('Improvement per Item (+ = better)')
ax3.set_xlabel('Item Index')
ax3.set_ylabel('Error Reduction')

plt.tight_layout()
plt.show()

In [ ]:
# Detailed comparison table
comparison_df = success_df[[
    'order_id', 'actual_weight_kg', 'old_ai_weight_kg', 'new_weight_kg',
    'old_error_rate', 'new_error_rate', 'reason'
]].copy()

comparison_df['improved'] = comparison_df['new_error_rate'] < comparison_df['old_error_rate']
comparison_df = comparison_df.round(3)

print(f"\nImproved: {comparison_df['improved'].sum()} / {len(comparison_df)} ({comparison_df['improved'].mean()*100:.0f}%)")
comparison_df

## 8. Export Results

In [ ]:
# Save to CSV
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'experiment_results_{timestamp}.csv'

results_df.to_csv(filename, index=False)
print(f"✅ Saved to {filename}")

# Download link
from google.colab import files
files.download(filename)

---

## Alternative Queries

Uncomment and run these cells to load different datasets.

In [ ]:
# # Query 3: Same Product, Multiple Orders (Uniformity Check)
# QUERY_UNIFORMITY = """
# WITH product_orders AS (
#   SELECT
#     oid.order_item_product_id,
#     oid.order_item_title_origin AS product_title,
#     oid.order_item_product_version_info_category AS category,
#     ARRAY_TO_STRING(oid.order_item_product_version_thumbnail_urls, '|') AS thumbnail_urls,
#     kse.actual_weight,
#     kse.dimensions,
#     kse.order_id,
#     COUNT(*) OVER (PARTITION BY oid.order_item_product_id) AS order_count
#   FROM `sazoshop.firestore_snapshot.v2_order_items` oid
#   INNER JOIN `sazoshop.firestore_collection.v2_kse_cost` kse 
#     ON oid.order_item_order_id = kse.order_id
#   WHERE
#     kse.actual_weight > 0 AND kse.actual_weight < 50
#     AND kse.dimensions IS NOT NULL
#     AND REGEXP_CONTAINS(kse.dimensions, r'^\\d+\\.?\\d*x\\d+\\.?\\d*x\\d+\\.?\\d*$')
#     AND oid.order_item_title_origin IS NOT NULL
# )
# SELECT 
#   order_item_product_id,
#   product_title,
#   category,
#   ANY_VALUE(thumbnail_urls) AS thumbnail_urls,
#   order_count,
#   AVG(actual_weight) AS avg_actual_weight,
#   STDDEV(actual_weight) AS stddev_actual_weight
# FROM product_orders
# WHERE order_count >= 3
# GROUP BY order_item_product_id, product_title, category, order_count
# ORDER BY order_count DESC
# LIMIT 50
# """
# 
# df_uniformity = client.query(QUERY_UNIFORMITY).to_dataframe()
# print(f"✅ Loaded {len(df_uniformity)} products for uniformity check")
# df_uniformity.head()